In [1]:
import pandas as pd
import numpy as np
import geopy.distance
import networkx as nx

# --- Airport Data Loading ---
try:
    # Assuming airports.csv from OurAirports is downloaded
    # Adjust path as needed
    airports_df = pd.read_csv('airports.csv')

    # Optional: Filter for relevant airports (e.g., large airports with IATA codes)
    airports_df = airports_df[
        airports_df['type'].isin(['large_airport', 'medium_airport']) &
        airports_df['iata_code'].notna()
    ].copy() # Use.copy() to avoid SettingWithCopyWarning

    # Create a dictionary for quick lookup: iata_code -> (latitude, longitude)
    # Ensure latitude/longitude are numeric
    airports_df['latitude_deg'] = pd.to_numeric(airports_df['latitude_deg'], errors='coerce')
    airports_df['longitude_deg'] = pd.to_numeric(airports_df['longitude_deg'], errors='coerce')
    airports_df.dropna(subset=['latitude_deg', 'longitude_deg'], inplace=True)

    airport_coords = {
        row['iata_code']: (row['latitude_deg'], row['longitude_deg'])
        for index, row in airports_df.iterrows()
    }
    print(f"Loaded data for {len(airport_coords)} airports.")
    
except FileNotFoundError:
    print("Error: airports.csv not found. Please download from OurAirports.")
    # Handle error appropriately, e.g., exit or use a fallback
    airport_coords = {} # Example fallback

# --- Aircraft Performance Data (Simplified) ---
# Using average values for a Boeing 737-800 as an example
# (Refine based on Table 2 and specific needs)
aircraft_params = {
    'avg_cruise_tas_knots': 453,  # knots (True Airspeed)
    'avg_fuel_burn_kg_hr': 2700, # kg/hour
}

Loaded data for 4489 airports.


In [ ]:
# Initialize the graph (Directed Graph recommended)
G = nx.DiGraph()

# Add nodes (airports)
for iata_code, coords in airport_coords.items():
    G.add_node(iata_code, pos=coords)

print(f"Added {G.number_of_nodes()} nodes to the graph.")

# Add edges (potential flight segments) - Fully Connected Approach Example
# Warning: This can be computationally intensive for many airports (~N^2 edges)
count = 0
max_edges_to_add = 100000 # Limit for demonstration, adjust as needed

nodes_list = list(G.nodes())# Get a list of nodes to iterate over
ex=[]
for i in range(len(nodes_list)):
    if count >= max_edges_to_add * 2 : # Check before inner loop starts fully
        print(f"Reached edge limit ({max_edges_to_add}). Stopping edge creation.")
        break
    for j in range(i + 1, len(nodes_list)):
        u = nodes_list[i]
        v = nodes_list[j]
        coords_u = G.nodes[u]['pos']
        coords_v = G.nodes[v]['pos']
        if j==1:
            ex.append([coords_u,coords_v])

        try:
            # Calculate geodesic distance in nautical miles
            distance_nm = geopy.distance.geodesic(coords_u, coords_v).nm

            # Estimate flight time in hours (using TAS as proxy for GS)
            # Avoid division by zero if speed is 0
            if aircraft_params['avg_cruise_tas_knots'] > 0:
                time_hours = distance_nm / aircraft_params['avg_cruise_tas_knots']
            else:
                time_hours = float('inf') # Or handle as error

            # Estimate fuel burn in kg
            fuel_kg = time_hours * aircraft_params['avg_fuel_burn_kg_hr']

            # Add edges in both directions (for DiGraph) with calculated weights
            # Ensure weights are non-negative
            if distance_nm >= 0 and time_hours >= 0 and fuel_kg >= 0:
                G.add_edge(u, v, distance=distance_nm, time=time_hours, fuel=fuel_kg)
                G.add_edge(v, u, distance=distance_nm, time=time_hours, fuel=fuel_kg)
                count += 2
            else:
                 print(f"Skipping edge between {u} and {v} due to invalid calculated weight.")


        except ValueError as e:
            # Handle potential errors in distance calculation (e.g., invalid coords)
            print(f"Could not calculate distance between {u} and {v}: {e}")
            continue # Skip this edge pair

        if count >= max_edges_to_add * 2:
             print(f"Reached edge limit ({max_edges_to_add}). Stopping edge creation.")
             break # Break inner loop

print(ex)
print(f"Added {G.number_of_edges()} edges to the graph.")

# Alternative: Route-Based Approach
# If using OpenFlights routes.dat:
# 1. Load routes data into a pandas DataFrame.
# 2. Iterate through routes. For each route (source_iata, dest_iata):
# 3. Check if source_iata and dest_iata are in airport_coords.
# 4. Calculate distance, time, fuel as above.
# 5. Add a single directed edge: G.add_edge(source_iata, dest_iata,...)

In [8]:
def find_optimal_path(graph, departure_iata, arrival_iata, cost_metric='time'):
    """
    Finds the optimal path using NetworkX shortest_path (Dijkstra).

    Args:
        graph (nx.DiGraph): The NetworkX graph with weighted edges.
        departure_iata (str): IATA code of the departure airport.
        arrival_iata (str): IATA code of the arrival airport.
        cost_metric (str): The edge attribute to use as weight ('time' or 'fuel').

    Returns:
        tuple: (list of nodes in the path, total cost) or (None, None) if no path.
    """
    if departure_iata not in graph:
        print(f"Error: Departure airport {departure_iata} not found in graph.")
        return None, None
    if arrival_iata not in graph:
        print(f"Error: Arrival airport {arrival_iata} not found in graph.")
        return None, None
    if cost_metric not in ['time', 'fuel', 'distance']:
         print(f"Error: Invalid cost_metric '{cost_metric}'. Use 'time', 'fuel', or 'distance'.")
         return None, None

    try:
        # Use nx.shortest_path and nx.shortest_path_length
        # Specify weight attribute corresponding to the cost metric
        optimal_path_nodes = nx.shortest_path(
            graph,
            source=departure_iata,
            target=arrival_iata,
            weight=cost_metric,
            method='dijkstra' # Explicitly use Dijkstra for non-negative weights
        )
        optimal_path_length = nx.shortest_path_length(
             graph,
             source=departure_iata,
             target=arrival_iata,
             weight=cost_metric,
             method='dijkstra'
        )

        # Alternatively, using Dijkstra directly:
        # optimal_path_length, optimal_path_nodes_dict = nx.single_source_dijkstra(
        #     graph, source=departure_iata, target=arrival_iata, weight=cost_metric
        # )
        # optimal_path_nodes = optimal_path_nodes_dict # If target is specified, it returns just the path

        return optimal_path_nodes, optimal_path_length

    except nx.NetworkXNoPath:
        print(f"No path found between {departure_iata} and {arrival_iata}.")
        return None, None
    except KeyError as e:
        print(f"Error accessing edge weight '{cost_metric}': {e}. Ensure edges have this attribute.")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred during pathfinding: {e}")
        return None, None


# --- Example Usage ---
dep_airport = 'CJB' # London Heathrow
arr_airport = 'JFK' # New York JFK
optimization_criterion = 'time' # Optimize for minimum time

path_nodes, path_cost = find_optimal_path(G, dep_airport, arr_airport, optimization_criterion)

if path_nodes:
    print(f"\nOptimal path ({optimization_criterion}) from {dep_airport} to {arr_airport}:")
    print(" -> ".join(path_nodes))
    print(f"Total estimated {optimization_criterion}: {path_cost:.2f} {'hours' if optimization_criterion == 'time' else 'kg'}")


Optimal path (time) from CJB to JFK:
CJB -> AXF -> JFK
Total estimated time: 18.34 hours


In [14]:
def calculate_path_metrics(graph, path_nodes):
    """
    Calculates total distance, time, and fuel for a given path.

    Args:
        graph (nx.DiGraph): The graph containing edge attributes.
        path_nodes (list): The sequence of nodes in the path.

    Returns:
        dict: A dictionary containing total_distance_nm, total_time_hours, total_fuel_kg.
              Returns None if the path is invalid or edges lack attributes.
    """
    if not path_nodes or len(path_nodes) < 2:
        return None

    total_distance = 0.0
    total_time = 0.0
    total_fuel = 0.0

    try:
        for i in range(len(path_nodes) - 1):
            u = path_nodes[i]
            v = path_nodes[i+1]
            edge_data = graph.edges[u, v] # Access edge attributes

            total_distance += edge_data.get('distance', 0) # Default to 0 if missing
            total_time += edge_data.get('time', 0)
            total_fuel += edge_data.get('fuel', 0)

        # Calculate mileage (avoid division by zero)
        mileage = total_distance / total_fuel if total_fuel > 0 else float('inf')

        return {
            'total_distance_nm': total_distance,
            'total_time_hours': total_time,
            'total_fuel_kg': total_fuel,
            'mileage_nm_per_kg': mileage
        }
    except KeyError as e:
         print(f"Error: Edge attribute missing for segment {u}-{v}: {e}")
         return None
    except Exception as e:
        print(f"An error occurred during metric calculation: {e}")
        return None


# --- Example Usage (Continuing from previous step) ---
if path_nodes:
    final_metrics = calculate_path_metrics(G, path_nodes)
    if final_metrics:
        print("\nCalculated Metrics for the Path:")
        print(f"  Total Distance: {final_metrics['total_distance_nm']:.2f} NM")
        print(f"  Total Journey Time: {final_metrics['total_time_hours']:.2f} hours")
        print(f"  Total Fuel Consumption: {final_metrics['total_fuel_kg']:.2f} kg")
        print(f"  Mileage: {final_metrics['mileage_nm_per_kg']:.2f} NM/kg")


Calculated Metrics for the Path:
  Total Distance: 8309.67 NM
  Total Journey Time: 18.34 hours
  Total Fuel Consumption: 49527.82 kg
  Mileage: 0.17 NM/kg
